In [ ]:
from libs import geo # Importing the intercept module
from libs import constants as cst  # Importing constants for speed conversion
from libs.transform import Transform  # Importing the Transform class for coordinate transformations
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize_scalar
import pymap3d as pm3d  # Importing pymap3d for geodetic to ECEF conversions
from libs import tacview  # Importing the tacview module for writing ACMI files

#Setup
#Target
lat_tgt= 34.71647 #degrees
lon_tgt= -117.8007 #degrees
alt_tgt= 10000*cst.FT_TO_M #m
hdg_tgt= 90.0 #degrees
TAS_tgt= 400*cst.KT_TO_MS #(0.7M at 10k ft)

#Fighter
lat_f= 35.0 #degrees
lon_f= -117.8907 #degrees
alt_f= 10000*cst.FT_TO_M #m
hdg_f= 0.0 #degrees
TAS_f= 510*cst.KT_TO_MS #knots (0.8M at 10k ft)
load_factor_f= 3.0 #g's

coord_tgt_init= Transform(lat=lat_tgt, lon=lon_tgt, alt=alt_tgt, roll=0, pitch=0, yaw=hdg_tgt)  # Create a Transform object for the target's initial position
coord_f_init= Transform(lat=lat_f, lon=lon_f, alt=alt_f, roll=0, pitch=0, yaw=hdg_f)  # Create a Transform object for the fighter's initial position
turn_radius_f= geo.turn_radius(TAS_f, load_factor_f)  # Calculate turn radius in meters
print (f"Turn radius: {turn_radius_f:.0f} m")
turn_rate_f = geo.turn_rate(TAS_f, load_factor_f)  # Calculate turn rate in rad/s
print (f"Turn rate: {turn_rate_f:.2f} °/s")



Turn radius: 2482 m
Turn rate: 6.06 °/s


In [ ]:

def constrained_total_time(init_turn, coord_tgt, coord_f, speed_tgt, speed_f, load_factor_f, tolerance,debug=False):
    try:
        total_time, CATA,_,_,_,_ = geo.intercept_traj(coord_tgt, coord_f, speed_tgt, speed_f, load_factor_f, init_turn,debug)
        if debug:
            print(f"init_turn={init_turn:.2f}°, total_time={total_time:.2f}s, CATA={CATA:.2f}°, error={geo.normalize_angle(CATA - (coord_f_init.yaw + init_turn)):.2f}°")
        if abs(geo.normalize_angle(CATA - (coord_f_init.yaw+init_turn))) > tolerance:
            return 1e6  # Penalize if constraint is violated
        return total_time + 1*abs(geo.normalize_angle(CATA - (coord_f_init.yaw+init_turn)))
    except:
        print(f"⚠️ Error with init_turn={init_turn:.2f}°")
        return 1e6  # Penalize if intercept_traj fails

def two_stage_turn_optimizer(coord_tgt, coord_f, speed_tgt, speed_f, load_factor_f, tolerance=5,debug=False):
    # --- Stage 1: Coarse search every 5 degrees
    coarse_angles = np.arange(-355, 355, 5)
    best_time = float('inf')
    best_angle = None
    times= []
    for angle in coarse_angles:
        t = constrained_total_time(angle, coord_tgt, coord_f, speed_tgt, speed_f, load_factor_f, tolerance,debug)
        if t < best_time:
            best_time = t
            best_angle = angle
            if debug:
                print(f"🔍 Coarse search improved: init_turn={angle:.2f}°, total_time={t:.2f}s")
        times.append((angle, t))
    if debug:
        print(f"🔍 Best coarse result: init_turn={best_angle:.2f}°, total_time={best_time:.2f}s")

    # --- Stage 2: Fine search ±5° around best_angle
    bounds = (best_angle - 5, best_angle + 5)
    tolerance=4 # Reduced tolerance for fine search
    result = minimize_scalar(
        constrained_total_time,
        args=(coord_tgt, coord_f, speed_tgt, speed_f, load_factor_f, tolerance,debug),
        bounds=bounds,
        method='bounded',
        options={'xatol': 0.1}
    )
    total_time, CATA,_,_,_,_ = geo.intercept_traj(coord_tgt, coord_f, speed_tgt, speed_f, load_factor_f, result.x)
    if debug:
        print(f"Result:{result}")
        print(f"CATA: {CATA:.2f}°, error: {geo.normalize_angle(CATA - (coord_f_init.yaw + result.x)):.2f}°")
        print(f"✅ Refined result: init_turn={result.x:.2f}°, total_time={result.fun:.4f}s")

    return result, times, CATA,total_time

def plot_total_time_vs_angle(times, refined_result=None):
    # Filter out penalty values (1e6)
    filtered_times = [(a, t) for a, t in times if t < 1e6]

    if not filtered_times:
        print("⚠️ No valid points to plot.")
        return

    angles, total_times = zip(*filtered_times)

    fig = go.Figure()

    # Main line plot
    fig.add_trace(go.Scatter(
        x=angles,
        y=total_times,
        mode='lines+markers',
        name='Coarse Sweep (valid)',
        line=dict(color='royalblue'),
        marker=dict(size=6)
    ))

    # Refined result marker (if available and valid)
    if refined_result and refined_result.fun < 1e6:
        fig.add_trace(go.Scatter(
            x=[refined_result.x],
            y=[refined_result.fun],
            mode='markers+text',
            name='Refined Minimum',
            marker=dict(color='red', size=10, symbol='star'),
            text=[f"{refined_result.fun:.2f}s"],
            textposition="top center"
        ))

    fig.update_layout(
        title='Intercept Total Time vs Initial Turn Angle',
        xaxis_title='Initial Turn (degrees)',
        yaxis_title='Total Time (seconds)',
        template='plotly_white',
        hovermode='x unified'
    )

    fig.show()

def get_trajectories(coord_tgt_init, coord_f_init, speed_tgt, speed_f, load_factor_f, init_turn):
    level=True #forces a level interception
    turn_rate_f = geo.turn_rate(speed_f, load_factor_f) 
    turn_radius_f = geo.turn_radius(speed_f, load_factor_f)  # Calculate turn radius in meters
    coord_f=[]
    coord_tgt=[]
    coord_f.append(coord_f_init)
    coord_tgt.append(coord_tgt_init)
    total_time, CATA, time_1, time_2, time_3,coord_coll = geo.intercept_traj(coord_tgt_init, coord_f_init, speed_tgt, speed_f, load_factor_f, init_turn)
    # print(f"Total time: {total_time:.2f} s, CATA: {CATA:.2f}°, init_turn: {init_turn:.2f}°, time_1: {time_1:.2f} s, time_2: {time_2:.2f} s, time_3: {time_3:.2f} s")
    #Generate time vector
    time_vector = np.arange(0, total_time, 1)  # Time vector in seconds
    time_vector = np.sort(np.unique(np.append(time_vector, [time_1, time_1+time_2, total_time]))) #add two extra points to ensure the trajectory is continuous
    #Generate target trajectory
    for t in time_vector:
        coord_tgt.append(geo.straight_line(coord_tgt_init, speed_tgt, t, level=level))  # Target trajectory
    
    #Generate fighter trajectory
    for i in range(int(time_1)):
        coord_f.append(geo.turn_exit(coord_f_init, (i+1)/time_1*init_turn, turn_radius_f, level=level))  # Fighter trajectory
    coord_f.append(geo.turn_exit(coord_f_init, init_turn, turn_radius_f, level=True))
    
    coord_f_1 = coord_f[-1]  # Store the last coordinate after the first turn    
    # print(f'Heading after first turn: {coord_f[-1].yaw:.2f}°, heading at f1: {coord_f_1.yaw:.2f}°, CATA: {CATA:.2f}°, THeorical: {init_turn:.2f}°')

    for i in range(int(time_2)):
        coord_f.append(geo.straight_line(coord_f_1, speed_f, (i+1), level=level))
    coord_f.append(geo.straight_line(coord_f_1, speed_f, time_2, level=level)) 
    
    # HCA=geo.normalize_angle(coord_tgt_init.yaw-coord_f[-1].yaw)  # Calculate the Heading Crossing Angle (HCA)

    # print(f"HCA: {HCA}°")
    coord_f_2= coord_f[-1]  # Store the last coordinate after the second turn
    coord_tgt_2= geo.straight_line(coord_tgt_init, speed_tgt, time_1+time_2)  # Extend the target trajectory to the end of the trajectory

    HCA=geo.get_HCA(coord_f_2,coord_tgt_2)

    for i in range(int(time_3)):
        coord_f.append(geo.turn_exit(coord_f_2, HCA*(i+1)/time_3, turn_radius_f, level=level))
    coord_f.append(geo.turn_exit(coord_f_2, HCA, turn_radius_f, level=level))  # Final turn to intercept the target

    coord_TIP = geo.get_TIP(coord_tgt_2, speed_tgt, speed_f, load_factor_f, HCA)  # Calculate the intercept point
    coord_TIP.yaw = coord_tgt_init.yaw  # Set the yaw of the intercept point to the target yaw
    
    # print(f"coord_f_3: {coord_f[-1]}")


    return time_vector, coord_tgt, coord_f,coord_coll, coord_TIP,coord_f_1, coord_f_2

def plot_trajectories_on_map(coord_tgt, coord_f,coord_coll, coord_TIP,coord_f_1, coord_f_2):
    def extract_coords(coord_list):
        lats = [c.lat for c in coord_list]
        lons = [c.lon for c in coord_list]
        return lats, lons

    tgt_lats, tgt_lons = extract_coords(coord_tgt)
    f_lats, f_lons = extract_coords(coord_f)

    fig = go.Figure()

    # Target trajectory
    fig.add_trace(go.Scattermap(
        lat=tgt_lats,
        lon=tgt_lons,
        mode='lines+markers',
        name='Target',
        line=dict(width=1, color='orange'),
        marker=dict(size=5, color='orange')
    ))

    # Fighter trajectory
    fig.add_trace(go.Scattermap(
        lat=f_lats,
        lon=f_lons,
        mode='lines+markers',
        name='Fighter',
        line=dict(width=1, color='blue'),
        marker=dict(size=5, color='blue')
    ))

    # Start markers
    fig.add_trace(go.Scattermap(lat=[tgt_lats[0]],lon=[tgt_lons[0]],mode='markers+text',name='Target Start',marker=dict(size=8, color='orange'),text=['Target Start'],))
    fig.add_trace(go.Scattermap(lat=[f_lats[0]],lon=[f_lons[0]],mode='markers+text',name='Fighter Start',marker=dict(size=8, color='blue'),text=['Fighter Start'],))
    fig.add_trace(go.Scattermap(lat=[coord_f_1.lat],lon=[coord_f_1.lon],mode='markers+text',name='Fighter Start',marker=dict(size=8, color='black'),text=['Fighter F1'],))
    fig.add_trace(go.Scattermap(lat=[coord_f_2.lat],lon=[coord_f_2.lon],mode='markers+text',name='Fighter Start',marker=dict(size=8, color='black'),text=['Fighter F2'],))
    fig.add_trace(go.Scattermap(lat=[coord_coll.lat], lon=[coord_coll.lon], mode='markers+text', marker=dict(size=8, color='black'), text=['Collision Point'],))
    fig.add_trace(go.Scattermap(lat=[coord_TIP.lat], lon=[coord_TIP.lon], mode='markers+text', marker=dict(size=8, color='purple'), text=['TIP Point'],))

    # Center map around midpoint of both tracks
    center_lat = (tgt_lats[0] + f_lats[0]) / 2
    center_lon = (tgt_lons[0] + f_lons[0]) / 2


    fig.update_layout(
        title='Fighter and Target Trajectories on Map',
        map=dict(
            style='satellite',  # or 'carto-positron', 'stamen-terrain', etc.
            center=dict(lat=center_lat, lon=center_lon),
            zoom=10
        ),
        height=700,
        margin=dict(l=0, r=0, t=40, b=0)
    )

    fig.show()


In [3]:
#Debug and display

Test=False
iteration=100

coord_tgt_init.yaw=90
coord_f_init.yaw=0
# coord_tgt_init.yaw=352.89 #close but not optimum solution
# coord_f_init.yaw=222.02


if Test:
    worst_error=0
    wc_tgt_hdg=0
    wc_f_hdg=0
    for i in range(iteration):
        debug=False
        tolerance=10
        # coord_tgt_init.yaw=9.2+5*np.random.rand()
        # coord_f_init.yaw=17.2+np.random.rand()
        coord_tgt_init.yaw=np.random.rand()*360
        coord_f_init.yaw=np.random.rand()*360
        res,times, CATA,total_time = two_stage_turn_optimizer(coord_tgt_init, coord_f_init, TAS_tgt, TAS_f, load_factor_f,tolerance,debug)
        # plot_total_time_vs_angle(times, refined_result=res)

        # plot_trajectories(time_vec, coord_tgt_traj, coord_f_traj)
        time_vec, coord_tgt_traj, coord_f_traj,coord_coll,coord_TIP,coord_f_1,coord_f_2 = get_trajectories(coord_tgt_init, coord_f_init, TAS_tgt, TAS_f, load_factor_f, res.x)

        dist_tgt=TAS_tgt*cst.KT_TO_MS*total_time  # Distance traveled by the target in meters
        distance_tgt_2d= geo.distance3D(coord_tgt_traj[0].lat,coord_tgt_traj[0].lon,coord_tgt_traj[0].alt, coord_tgt_traj[-1].lat,coord_tgt_traj[-1].lon,coord_tgt_traj[-1].alt)  # 2D distance to the intercept point
        HCA=geo.normalize_angle(coord_tgt_init.yaw-coord_f_2.yaw)  # Calculate the Heading Crossing Angle (HCA)
        dist_f_0_1=abs(res.x) * np.pi/180*turn_radius_f 
        dist_f_1_2=geo.distance3D(coord_f_1.lat,coord_f_1.lon,coord_f_1.alt,coord_f_2.lat,coord_f_2.lon,coord_f_2.alt)
        dist_f_2_3 = abs(HCA)* np.pi/180*turn_radius_f # Distance traveled by the fighter in meters
        dist_f=TAS_f*cst.KT_TO_MS*total_time
        dist_f_tot= dist_f_0_1 + dist_f_1_2 + dist_f_2_3  # Total distance traveled by the fighter in meters
        # print(f"Total time: {total_time:.2f} s, CATA: {CATA:.2f}°, init_turn: {res.x:.2f}°, distance_tgt time: {dist_tgt:.2f} m, distance_tgt_traj: {distance_tgt_2d:.2f} m, distance_f time: {dist_f:.2f} m, distance_f_traj: {distance_f_traj:.2f} m")
        # print(f"Total time: {total_time:.2f} s, distance_f_0_1: {dist_f_0_1:.2f} m, distance_f_1_2: {dist_f_1_2:.2f} m, distance_f_2_3: {dist_f_2_3:.2f} m, distance_f_tot: {dist_f_tot:.2f} m")
        # print(f"Distance f time: {dist_f:.2f} m ")
        # print(f"HCA:{HCA:.2f}°")
        times_key=[t for t in time_vec if not float(t).is_integer()]
        # print(times_key)
        coord_tgt_1=geo.straight_line(coord_tgt_init, TAS_tgt, times_key[0])
        coord_tgt_2= geo.straight_line(coord_tgt_init, TAS_tgt, times_key[1])  # Extend the target trajectory to the end of the trajectory
        coord_TIP_2=geo.get_TIP(coord_tgt_2,TAS_tgt,TAS_f,load_factor_f,HCA)
        coord_TIPs=geo.get_TIPs(coord_tgt_2,TAS_tgt,TAS_f,load_factor_f)

        error_2D=geo.distance2D(coord_f_traj[-1].lat,coord_f_traj[-1].lon,coord_tgt_traj[-1].lat,coord_tgt_traj[-1].lon)
        error_HDG=geo.normalize_angle(coord_f_traj[-1].yaw-coord_tgt_traj[-1].yaw)
        print(f"TGT hdg:{coord_tgt_init.yaw:.1f}°\tFighter hdg:{coord_f_init.yaw:.1f}°\tHCA:{HCA:.1f}°\tCATA:{CATA:.1f}°\tHorinzontal error:{error_2D:.0f} m\tHdg error:{error_HDG:.1f}°\tTime:{total_time:.0f} s")
        if error_2D>worst_error:
            worst_error=error_2D
            wc_tgt_hdg=coord_tgt_init.yaw
            wc_f_hdg=coord_f_init.yaw
    print(f"❌ Worst case:")
    print(f"Error: {worst_error:.0f} m\ttgt HDG: {wc_tgt_hdg:.2f}°\tfighter HDG: {wc_f_hdg:.2f}°")

else:
    debug=True
    tolerance=5
    res,times, CATA,total_time = two_stage_turn_optimizer(coord_tgt_init, coord_f_init, TAS_tgt, TAS_f, load_factor_f,tolerance,debug)
    # plot_total_time_vs_angle(times, refined_result=res)

    # plot_trajectories(time_vec, coord_tgt_traj, coord_f_traj)
    time_vec, coord_tgt_traj, coord_f_traj,coord_coll,coord_TIP,coord_f_1,coord_f_2 = get_trajectories(coord_tgt_init, coord_f_init, TAS_tgt, TAS_f, load_factor_f, res.x)

    dist_tgt=TAS_tgt*cst.KT_TO_MS*total_time  # Distance traveled by the target in meters
    distance_tgt_2d= geo.distance3D(coord_tgt_traj[0].lat,coord_tgt_traj[0].lon,coord_tgt_traj[0].alt, coord_tgt_traj[-1].lat,coord_tgt_traj[-1].lon,coord_tgt_traj[-1].alt)  # 2D distance to the intercept point

    times_key=[t for t in time_vec if not float(t).is_integer()]
    coord_tgt_1=geo.straight_line(coord_tgt_init, TAS_tgt, times_key[0])
    coord_tgt_2= geo.straight_line(coord_tgt_init, TAS_tgt, times_key[1])  # Extend the target trajectory to the end of the trajectory

    HCA=geo.get_HCA(coord_f_2,coord_tgt_2)
    coord_TIP_2=geo.get_TIP(coord_tgt_2,TAS_tgt,TAS_f,load_factor_f,HCA)
    coord_TIPs=geo.get_TIPs(coord_tgt_2,TAS_tgt,TAS_f,load_factor_f)
    # HCA=geo.normalize_angle(coord_tgt_init.yaw-coord_f_2.yaw)  # Calculate the Heading Crossing Angle (HCA)
    dist_f_0_1=abs(res.x) * np.pi/180*turn_radius_f 
    dist_f_1_2=geo.distance3D(coord_f_1.lat,coord_f_1.lon,coord_f_1.alt,coord_f_2.lat,coord_f_2.lon,coord_f_2.alt)
    dist_f_2_3 = abs(HCA)* np.pi/180*turn_radius_f # Distance traveled by the fighter in meters
    dist_f=TAS_f*cst.KT_TO_MS*total_time
    dist_f_tot= dist_f_0_1 + dist_f_1_2 + dist_f_2_3  # Total distance traveled by the fighter in meters
    # print(f"Total time: {total_time:.2f} s, CATA: {CATA:.2f}°, init_turn: {res.x:.2f}°, distance_tgt time: {dist_tgt:.2f} m, distance_tgt_traj: {distance_tgt_2d:.2f} m, distance_f time: {dist_f:.2f} m, distance_f_traj: {distance_f_traj:.2f} m")
    # print(f"Total time: {total_time:.2f} s, distance_f_0_1: {dist_f_0_1:.2f} m, distance_f_1_2: {dist_f_1_2:.2f} m, distance_f_2_3: {dist_f_2_3:.2f} m, distance_f_tot: {dist_f_tot:.2f} m")
    # print(f"Distance f time: {dist_f:.2f} m ")
    # print(f"HCA:{HCA:.2f}°")
    # print(times_key)


    error_2D=geo.distance2D(coord_f_traj[-1].lat,coord_f_traj[-1].lon,coord_tgt_traj[-1].lat,coord_tgt_traj[-1].lon)
    error_HDG=geo.normalize_angle(coord_f_traj[-1].yaw-coord_tgt_traj[-1].yaw)
    print(f"TGT hdg:{coord_tgt_init.yaw:.1f}°\tFighter hdg:{coord_f_init.yaw:.1f}°\tHCA:{HCA:.1f}°\tCATA:{CATA:.1f}°\tHorinzontal error:{error_2D:.0f} m\tHdg error:{error_HDG:.1f}°\tTime:{total_time:.0f} s")


CATA: 105.8°, TGT HDG:90.0°, HCA: 85.0°
init_turn=-355.00°, total_time=523.59s, CATA=106.34°, error=101.34°
🔍 Coarse search improved: init_turn=-355.00°, total_time=1000000.00s
CATA: 105.8°, TGT HDG:90.0°, HCA: 80.0°
init_turn=-350.00°, total_time=515.91s, CATA=106.35°, error=96.35°
CATA: 105.8°, TGT HDG:90.0°, HCA: 75.0°
init_turn=-345.00°, total_time=508.81s, CATA=106.35°, error=91.35°
CATA: 105.8°, TGT HDG:90.0°, HCA: 70.0°
init_turn=-340.00°, total_time=502.28s, CATA=106.32°, error=86.32°
CATA: 105.7°, TGT HDG:90.0°, HCA: 65.0°
init_turn=-335.00°, total_time=496.34s, CATA=106.28°, error=81.28°
CATA: 105.7°, TGT HDG:90.0°, HCA: 60.0°
init_turn=-330.00°, total_time=490.97s, CATA=106.22°, error=76.22°
CATA: 105.7°, TGT HDG:90.0°, HCA: 55.0°
init_turn=-325.00°, total_time=486.17s, CATA=106.15°, error=71.15°
CATA: 105.6°, TGT HDG:90.0°, HCA: 50.0°
init_turn=-320.00°, total_time=481.92s, CATA=106.07°, error=66.07°
CATA: 105.6°, TGT HDG:90.0°, HCA: 45.0°
init_turn=-315.00°, total_time=478

In [4]:
#plot trajectory
def extract_coords(coord_list):
        lats = [c.lat for c in coord_list]
        lons = [c.lon for c in coord_list]
        return lats, lons

tgt_lats, tgt_lons = extract_coords(coord_tgt_traj)
f_lats, f_lons = extract_coords(coord_f_traj)
TIPs_lat,TIPs_lon=extract_coords(coord_TIPs)

fig = go.Figure()

# Target trajectory
fig.add_trace(go.Scattermap(
    lat=tgt_lats,
    lon=tgt_lons,
    mode='lines+markers',
    name='Target',
    line=dict(width=1, color='orange'),
    marker=dict(size=5, color='orange')
))

# Fighter trajectory
fig.add_trace(go.Scattermap(
    lat=f_lats,
    lon=f_lons,
    mode='lines+markers',
    name='Fighter',
    line=dict(width=1, color='blue'),
    marker=dict(size=5, color='blue')
))

# Start markers
fig.add_trace(go.Scattermap(lat=[tgt_lats[0]],lon=[tgt_lons[0]],mode='markers+text',name='Target Start',marker=dict(size=8, color='orange'),text=['Target Start'],))
fig.add_trace(go.Scattermap(lat=[f_lats[0]],lon=[f_lons[0]],mode='markers+text',name='Fighter Start',marker=dict(size=8, color='blue'),text=['Fighter Start'],))
fig.add_trace(go.Scattermap(lat=[coord_f_1.lat],lon=[coord_f_1.lon],mode='markers+text',name='Fighter Start',marker=dict(size=8, color='black'),text=['Fighter F1'],))
fig.add_trace(go.Scattermap(lat=[coord_f_2.lat],lon=[coord_f_2.lon],mode='markers+text',name='Fighter Start',marker=dict(size=8, color='black'),text=['Fighter F2'],))
fig.add_trace(go.Scattermap(lat=[coord_coll.lat], lon=[coord_coll.lon], mode='markers+text', marker=dict(size=8, color='black'), text=['Collision Point'],))
fig.add_trace(go.Scattermap(lat=[coord_TIP_2.lat], lon=[coord_TIP_2.lon], mode='markers+text', marker=dict(size=8, color='purple'), text=['TIP 2 Point'],))
fig.add_trace(go.Scattermap(lat=[coord_tgt_1.lat], lon=[coord_tgt_1.lon], mode='markers+text', marker=dict(size=12, color='red'), text=['Tgt 1'],))
fig.add_trace(go.Scattermap(lat=[coord_tgt_2.lat], lon=[coord_tgt_2.lon], mode='markers+text', marker=dict(size=12, color='red'), text=['Tgt 2'],))
fig.add_trace(go.Scattermap(lat=[coord_TIP.lat], lon=[coord_TIP.lon], mode='markers+text', marker=dict(size=8, color='purple'), text=['TIP Point'],))

#TIP with custom template
TIPs_customdata = [[round(c.yaw, 1), round(c.lat, 5), round(c.lon, 5)] for c in coord_TIPs]
fig.add_trace(go.Scattermap(lat=TIPs_lat, lon=TIPs_lon, name='TIPs 2',mode='markers+text',
            marker=dict(size=8, color='black'),
            text=['Tips'],
            customdata=TIPs_customdata,
            hovertemplate=(
            "HCA: %{customdata[0]}°<br>"
            "Lat: %{customdata[1]}<br>"
            "Lon: %{customdata[2]}<extra></extra>"
            ))
)







# Center map around midpoint of both tracks
center_lat = coord_coll.lat
center_lon = coord_coll.lon


fig.update_layout(
    title='Fighter and Target Trajectories on Map',
    map=dict(
        style='satellite',  # or 'carto-positron', 'stamen-terrain', etc.
        center=dict(lat=center_lat, lon=center_lon),
        zoom=10
    ),
    height=700,
    margin=dict(l=0, r=0, t=40, b=0)
)

fig.show()